In [2]:
import ciao_contrib.runtool as rt
import sherpa.astro.ui as ui
import numpy as np
import matplotlib.pyplot as plt
import os

If all files use the same WCS coordinates, we can rename the region('blah') to just be that one single file, which would be helpful.

In [10]:
def extract_spec(obsid):
    '''
    Extract the spectrum from the evt2 file and save into the data folder
    with the name of the observation ID followed by GW170817.

    Parameters
    ----------
    obsid : int
        The observation ID of the data to be extracted

    Returns
    -------
    None
    '''
    spec = rt.specextract
    spec.punlearn()
    spec.infile = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits[sky=region(data/{obsid}/repro/srcwcs.reg)]'
    spec.bkgfile = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits[sky=region(data/{obsid}/repro/bkgwcs.reg)]'
    spec.outroot = f'data/{obsid}/GW170817/GW170817'
    spec.correctpsf = 'yes'
    spec.weight = 'no'
    spec()

In [37]:
extract_spec(20861)

In [2]:
rt.celldetect

<CIAO tool: celldetect>

In [55]:
# Run detect on the evt2 file to create a source list using the celldetect method

def detect_srcs(obsid):

    '''
    Run the celldetect method on the evt2 file to create a source list.

    Parameters
    ----------
    obsid : int
        The observation ID of the data to be extracted

    Returns
    -------
    None
    '''

    detect = rt.celldetect
    detect.punlearn()
    detect.infile = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
    detect.outfile = f'data/{obsid}/GW170817/GW170817_src_6.fits'
    detect.fixedcell = 6
    detect()

def detect_srcs_wav(obsid):

    detect = rt.wavdetect
    detect.punlearn()
    detect.infile = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
    detect.outfile = f'data/{obsid}/GW170817/GW170817_src_wav9.fits'
    detect.scellfile = f'data/{obsid}/GW170817/GW170817_scell.fits'
    detect.imagefile = f'data/{obsid}/GW170817/GW170817_img.fits'
    detect.clobber = 'yes'
    detect.sigthresh = 1e-9
    detect.defnbkgfile = f'data/{obsid}/GW170817/GW170817_defnbkg.fits'
    detect()

In [56]:
detect_srcs(20860)

In [57]:
detect_srcs(20936)

In [23]:
# Create GW170817 directory in data/{obsid}

def create_GWdir(obsid):
    
        '''
        Create a directory for the GW170817 data.
    
        Parameters
        ----------
        obsid : int
            The observation ID of the data to be extracted
    
        Returns
        -------
        None
        '''
    
        os.mkdir(f'data/{obsid}/GW170817')

In [13]:
create_GWdir(20861)


detect_srcs(20861)

In [61]:
def reproject(obsid_in, obsid_ref):

    reproj = rt.reproject_aspect
    reproj.punlearn()
    reproj.infile = f'data/{obsid_in}/GW170817/GW170817_src_6.fits'
    reproj.refsrcfile = f'data/{obsid_ref}/GW170817/GW170817_src_6.fits'
    reproj.updfile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    reproj.wcsfile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    reproj.residlim = 0.5
    reproj.outfile = f'data/{obsid_in}/GW170817/GW170817_reproj.fits'
    reproj()

THIS WORKS

In [62]:
reproject(20936, 20860)

In [30]:
def src_reproject(obsid_in, obsid_ref):
    ''' 
    Create a source list and then reproject to match the reference obsid.
    '''
    if not os.path.exists(f'data/{obsid_in}/GW170817'):
        create_GWdir(obsid_in)

    detect_srcs_wav(obsid_in)

    reproject(obsid_in, obsid_ref)

In [31]:
src_reproject(20936, 20860)

KeyboardInterrupt: 

Below here is useless? 11 PM Tuesday 4/25

In [40]:
detect_srcs_wav(20860)

In [41]:
detect_srcs_wav(20936)

In [53]:
def reproject_wav(obsid_in, obsid_ref):

    reproj = rt.reproject_aspect
    reproj.punlearn()
    reproj.infile = f'data/{obsid_in}/GW170817/GW170817_src_wav9.fits'
    reproj.refsrcfile = f'data/{obsid_ref}/GW170817/GW170817_src_wav9.fits'
    reproj.updfile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    reproj.wcsfile = f'data/{obsid_ref}/repro/acisf{obsid_ref}_repro_evt2.fits'
    reproj.residlim = 2
    reproj.outfile = f'data/{obsid_in}/GW170817/GW170817_reproj.fits'
    reproj()

In [54]:
reproject_wav(20936, 20860)

In [10]:
def merge_observations(obsid_in_list):
    '''
    params 
    ======
    obsid_in_list
        a list of integer obsid numbers
    
    output
    ======
    None
    '''

    flxobs = rt.flux_obs
    flxobs.punlearn()
    flxobs.infiles = str()
    for obsid in obsid_in_list:
        if flxobs.infiles is None:
            flxobs.infiles = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
        else:
            flxobs.infiles = flxobs.infiles+', '+f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
    print(flxobs.infiles)
    flxobs.outroot = 'data/merge_test/'
    flxobs()


##  Example function call
merge_observations([20860,20861])


data/20860/repro/acisf20860_repro_evt2.fits,data/20861/repro/acisf20861_repro_evt2.fits
